In [2]:
# ─── 0) Montar Drive y autenticar (Colab) ─────────────────────────────────
from google.colab import drive, auth
drive.mount('/content/drive')
auth.authenticate_user()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os, glob, re
import pandas as pd
from datetime import datetime
from google.cloud import bigquery

# ─── Configuración ────────────────────────────────────────────────────────
PROJECT_ID = "pokemon-cards-project"
DATASET_ID = "pokemon_dataset"
CSV_FOLDER = "/content/drive/MyDrive/Proyecto API/MonthlyCSV"
RENAMED_PREFIX = "monthly_price_"

client = bigquery.Client(project=PROJECT_ID)

# ─── 1) Renombrar archivos con fecha (cualquier prefijo) ─────────────────────
for path in glob.glob(f"{CSV_FOLDER}/*.csv"):
    # Buscar fecha en cualquier parte del nombre (YYYY-MM-DD)
    match = re.search(r"(\d{4})-(\d{2})-(\d{2})", os.path.basename(path))
    if not match:
        print(f"⚠️ No se detecta fecha en: {os.path.basename(path)}")
        continue

    # Extraer y reformatear la fecha
    date_str = f"{match.group(1)}{match.group(2)}{match.group(3)}"
    new_name = f"monthly_price_{date_str}.csv"
    new_path = os.path.join(CSV_FOLDER, new_name)

    # Solo renombrar si no existe aún
    if not os.path.exists(new_path):
        os.rename(path, new_path)
        print(f"🔁 Renombrado: {os.path.basename(path)} → {new_name}")
    else:
        print(f"⏩ Ya existe renombrado: {new_name}")

# ─── 2) Consultar tablas existentes ───────────────────────────────────────
existing_tables = [t.table_id for t in client.list_tables(DATASET_ID) if t.table_id.startswith("monthly_")]
print(f"ℹ️ Tablas mensuales ya en BigQuery: {len(existing_tables)}")

# ─── 3) Procesar y subir archivos nuevos ──────────────────────────────────
all_meta = []
for path in sorted(glob.glob(f"{CSV_FOLDER}/{RENAMED_PREFIX}*.csv")):
    match = re.search(r"_(\d{8})\.csv$", path)
    if not match:
        print(f"⚠️ Nombre incorrecto: {path}")
        continue

    date_str = match.group(1)
    date_fmt = datetime.strptime(date_str, "%Y%m%d")
    table_name = f"monthly_{date_fmt.year}_{date_fmt.month:02d}_{date_fmt.day:02d}"
    if table_name in existing_tables:
        print(f"⏩ Ya existe {table_name}, salto...")
        continue

    df = pd.read_csv(path)

    # (A) Metadatos (solo una vez, usando el archivo más reciente)
    all_meta.append(df[[c for c in df.columns if not c.startswith("cm_") and not c.startswith("tcg_")]])

    # (B) Precios
    price_df = df[["id", "name", "set_id", "set_name"] + [c for c in df.columns if c.startswith("cm_") or c.startswith("tcg_")]]

    job = client.load_table_from_dataframe(
        price_df,
        f"{PROJECT_ID}.{DATASET_ID}.{table_name}",
        job_config=bigquery.LoadJobConfig(
            autodetect=True,
            write_disposition="WRITE_TRUNCATE",
            create_disposition="CREATE_IF_NEEDED"
        )
    )
    job.result()
    print(f"✅ Subido {table_name} ({len(price_df)} filas)")

# ─── 4) Subir metadatos únicos ────────────────────────────────────────────
if all_meta:
    df_meta_all = pd.concat(all_meta).drop_duplicates(subset="id").reset_index(drop=True)
    table_meta = f"{PROJECT_ID}.{DATASET_ID}.card_metadata"
    job = client.load_table_from_dataframe(
        df_meta_all,
        table_meta,
        job_config=bigquery.LoadJobConfig(
            autodetect=True,
            write_disposition="WRITE_TRUNCATE",
            create_disposition="CREATE_IF_NEEDED"
        )
    )
    job.result()
    print(f"✅ Metadatos subidos: {len(df_meta_all)} cartas")
else:
    print("⚠️ No se encontraron metadatos para subir.")


⚠️ No se detecta fecha en: monthly_price_20250401.csv
⚠️ No se detecta fecha en: monthly_price_20250430.csv
⚠️ No se detecta fecha en: prices_2025_04.csv
ℹ️ Tablas mensuales ya en BigQuery: 0
✅ Subido monthly_2025_04_01 (18876 filas)
✅ Subido monthly_2025_04_30 (18876 filas)
✅ Metadatos subidos: 18876 cartas
